<a href="https://colab.research.google.com/github/LeonardoGoncRibeiro/05_AppliedMachineLearning/blob/main/08_MLOps_Deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLOps: Model deploy

In this course, we will learn how to deploy our model in a way that other people can have easy access to its predictions. We will understand how to create a server for our model, and how to use App Engine. Also, we will see how to combine our app with Docker, and how to do automatic deploy with GitHub Actions.

Note that, in this course, this notebook serves more as a "class notes" than a Python console, since very few things we be run in Python. Also, this is a continuation of the previous course, which showed how to develop a local API for our model. 

So, let's get the API we built in the previous course:

In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 14.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=d99666bcfdced67d190bc25e2fc71699f927b05a05697a792b34eb07c1bb0109
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [2]:
!pip install flask-basicauth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for flask-basicauth: filename=Flask_BasicAuth-0.2.0-py3-none-any.whl size=4243 sha256=374ae329f8b15685abfe3ac5bf702e796b620d312be3476f2ea0ac116a59e020
  Stored in directory: /root/.cache/pip/wheels/d5/08/a3/19638d90fdf01258ede772449bcbde424839459749acb977b6
Successfully built flask-basicauth


In [ ]:
import pandas as pd

from flask import Flask
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask_basicauth import BasicAuth

import pickle

my_first_app = Flask(__name__)
run_with_ngrok(my_first_app)

model_LinReg = pickle.load(open('model_LinReg.sav', 'rb'))

X_cols = ['tamanho', 'ano', 'garagem']

port = 5000
auth_token = '2ATxMOi0B3lpLciSMzBb9eOAejh_7uTTU7DVWwQH18eA3RnkQ'
ngrok.set_auth_token(auth_token)
public_url = ngrok.connect(port).public_url

my_first_app.config['BASIC_AUTH_USERNAME'] = 'Leo'
my_first_app.config['BASIC_AUTH_PASSWORD'] = 'pswd'
basic_auth = BasicAuth(my_first_app)

@my_first_app.route('/')
def home( ):
  return "My first API."

@my_first_app.route('/sentiment/<text>')
@basic_auth.required
def sentiment(text):
  tb_pt = TextBlob(text)
  tb_en = tb_pt.translate(from_lang = 'pt', to = 'en')
  pol = tb_en.polarity
  return "polarity: {}".format(pol)

@my_first_app.route('/linreg/', methods  = ['POST'])    # Now, we are using the method POST to receive new entries
def linreg( ):
  x_pred = request.get_json( )
  input = [x_pred[col] for col in X_cols]
  y_pred = model_LinReg.predict([input])[0]
  return jsonify(price = y_pred)

my_first_app.run( )

## Project templates

To create our project, we can use a template created by other people. For that end, we can use the cookie cutter package:

https://cookiecutter.readthedocs.io/en/stable/

In [1]:
!pip install cookiecutter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 20.3 MB/s 
     |████████████████████████████████| 64 kB 2.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


A very well-known template from the community comes from the driven data github:

https://drivendata.github.io/cookiecutter-data-science/

To start a new project via command line, we can do:

``` cookiecutter https://github.com/drivendata/cookiecutter-data-science ```

After we run this command in the command line, we will get asked:

*   The project name.
*   The project repository.
*   The author's name.
*   A description of the project.
*   And others.

This will create a template project for us, where we can start to work inside. Then, we can do some modifications to include our API in the template. Some modifications we may need to do:

*   Copy the .py files to the *notebooks* folder.
*   Copy the data files to the *data* folder, and include it in the adequate folder.
*   Copy the API to the *src* folder, in a new folder named *app*.
*   Copy your serialized model to the *models* folder.
*   Copy the requirements.txt file (which can be created using ```pip freeze > requirements.txt```

After we make the modifications to our template, we can create a GitHub repository.








## Making small modifications on our code

So, before uploading our API in the GitHub repository, it is important that we make some small modifications:

*   The authentication user and password should not show in the code. Instead, we should use environmental variables to handle these.
*   We should get our serialized model from the correct folder.



In [ ]:
my_first_app = Flask(__name__)
run_with_ngrok(my_first_app)

model_LinReg = pickle.load(open('../../models/model_LinReg.sav', 'rb'))            # Changing the folder for the model file

X_cols = ['tamanho', 'ano', 'garagem']

port = 5000
auth_token = '2ATxMOi0B3lpLciSMzBb9eOAejh_7uTTU7DVWwQH18eA3RnkQ'
ngrok.set_auth_token(auth_token)
public_url = ngrok.connect(port).public_url

my_first_app.config['BASIC_AUTH_USERNAME'] = os.environ.get('BASIC_AUTH_USERNAME') # Getting the authentication via environmental variables
my_first_app.config['BASIC_AUTH_PASSWORD'] = os.environ.get('BASIC_AUTH_PASSWORD') # Getting the authentication via environmental variables
basic_auth = BasicAuth(my_first_app)

@my_first_app.route('/')
def home( ):
  return "My first API."

@my_first_app.route('/sentiment/<text>')
@basic_auth.required
def sentiment(text):
  tb_pt = TextBlob(text)
  tb_en = tb_pt.translate(from_lang = 'pt', to = 'en')
  pol = tb_en.polarity
  return "polarity: {}".format(pol)

@my_first_app.route('/linreg/', methods  = ['POST'])    
def linreg( ):
  x_pred = request.get_json( )
  input = [x_pred[col] for col in X_cols]
  y_pred = model_LinReg.predict([input])[0]
  return jsonify(price = y_pred)

my_first_app.run( host = '0.0.0.0' )                                               # Changing the host

## Creating a virtual environment and running our API

To make sure that the libraries employed in the model deployment are the same as the ones used by the end-user, we should always use virtual environments to deploy applications. We can do this using virtualenv. From the command line, we can create a virtual environment using:

``` virtualenv new_environment```

where we create a new virtual environment named *new_environment*. To activate our virtual environment:

``` source venv/bin/activate ```

Now, we should install our requirements on the virtual environment:

``` pip install -r requirements.txt ```

Then, we can create our environmental variables:

``` export BASIC_AUTH_USERNAME = Leo ```

``` export BASIC_AUTH_PASSWORD = pswd ```

Nice! Now, we can run our application from our virtual environment doing:

``` cd src/app/ ```

``` python main.py ```

## Uploading our repository to GitHub

So, to upload our repository to GitHub, we can do:

``` git init ```

``` git add . ```

``` git commit -m "first commit"```

``` git remote add origin (GIT_HUB REPOSITORY LINK) ```

``` git push -u origin master ```

# Using Google Cloud Platform

To be able to use other servers (instead of the local server), we can use the Google Cloud Platform. Using the Google Cloud Platform, we can create a server to rent a machine to keep our application running. 

To that end, we can go to:

> Google Compute Engine - VM Instances - Create

Here, we will create a virtual machine to run our API. We can even change the type of machine we are using. The better the machine, the higher the price. We should allow for HTTP and HTTPS traffic, and create our machine.

## Unlocking the port from the virtual machine

After we create our VM, we can go to the settings, see network details, network default, and change the firewall rules. We should change the TCP Port to the Port 5000, which is used by our API.

## Deploying our application on Compute Engine

Now, we just connect to the machine using the SSH button. Now, we should install our app code and our softwares inside our virtual environment. So, we can do:

``` sudo apt-get update ```

``` sudo apt-get install git-all ```

``` git clone (GIT REPOSITORY LINK) ```

``` sudo apt-get python3 ```

``` pip3 install virtualenv --user ```

``` virtualenv venv ```

```venv/bin/activate ```

``` pip3 install -r requirements.txt ```

``` export BASIC_AUTH_USERNAME = Leo ```

``` export BASIC_AUTH_PASSWORD = pswd ```

And, to run our API, we should just call for the URL:

> (Virtual Machine IP):5000

Nice! Now, our API can be assessed from any machine, while we keep our virtual machine running (and paying for it)

# Serverless application using App Engine

We can also create a serverless application using App Engine, where we do not have to worry about managing our server. 

First, we should creae an App Engine configuration file:

```
runtime: python
env: flex
entrypoint: gunicorn -b :$PORT main:app
runtime_config:
  python_version: 3

includes: 
- env_vars.yaml
```

The environmental variables can be stated in another file, and this file can be included in the .gitignore file:

```
env-variables:
  BASIC_AUTH_USERNAME: Leo
  BASIC_AUTH_PASSWORD: pswd
```


We should also update our API:

In [ ]:
my_first_app = Flask(__name__)
run_with_ngrok(my_first_app)

model_LinReg = pickle.load(open('../../models/model_LinReg.sav', 'rb'))            

X_cols = ['tamanho', 'ano', 'garagem']

port = 5000
auth_token = '2ATxMOi0B3lpLciSMzBb9eOAejh_7uTTU7DVWwQH18eA3RnkQ'
ngrok.set_auth_token(auth_token)
public_url = ngrok.connect(port).public_url

my_first_app.config['BASIC_AUTH_USERNAME'] = os.environ.get('BASIC_AUTH_USERNAME') 
my_first_app.config['BASIC_AUTH_PASSWORD'] = os.environ.get('BASIC_AUTH_PASSWORD') 
basic_auth = BasicAuth(my_first_app)

@my_first_app.route('/')
def home( ):
  return "My first API."

@my_first_app.route('/sentiment/<text>')
@basic_auth.required
def sentiment(text):
  tb_pt = TextBlob(text)
  tb_en = tb_pt.translate(from_lang = 'pt', to = 'en')
  pol = tb_en.polarity
  return "polarity: {}".format(pol)

@my_first_app.route('/linreg/', methods  = ['POST'])    
def linreg( ):
  x_pred = request.get_json( )
  input = [x_pred[col] for col in X_cols]
  y_pred = model_LinReg.predict([input])[0]
  return jsonify(price = y_pred)

if __name__ == '__main__':    # Adding this condition to not allow multiple unnecessary runs
  my_first_app.run( host = '0.0.0.0' )    

Now, to deploy our API, we can run in the terminal:

``` gcloud app deploy ```

# Docker containers

A container Docker can be downloaded from:

> docker.com

Basically, we have to install docker and create a file on our template named DockerFile. So, in this DockerFile, we can deploy our application:

```
FROM python:3.7-slim

ARG BASIC_AUTH_USERNAME_ARG
ARG BASIC_AUTH_PASSWORD_ARG

ENV BASIC_AUTH_USERNAME = $BASIC_AUTH_USERNAME_ARG
ENV BASIC_AUTH_PASSWORD = $BASIC_AUTH_PASSWORD_ARG

COPY ./requirements.txt /usr/requirements.txt

WORKDIR /usr

RUN pip3 install -r requirements.txt

COPY ./src /usr/src
COPY ./models /usr/models

ENTRYPOINT ["python3"]

CMD ["src/app/main.py"]
```

Now, to create our docker image, we can run:

``` 
docker build -t ml-api --build-arg BASIC_AUTH_USERNAME_ARG = Leo --build-arg BASIC_AUTH_PASSWORD_ARG = pswd

docker run -p 5000:5000 ml-apli
```

To use a Serverless Docker container, we can use the Cloud Run API on Google Cloud Platform. We should user the Container Registry API. Then, we run in the command line:

``` gcloud auth configure-docker ```

Then, we can do:

``` docker tag (app-name) gcr.io/(project-name)/(docker-image-name) ```

Now, we can access Cloud Run from GCP, use our service (project-name) and create.